# Comprehensive SyftBox Permissions Tutorial
## Part 2: Patterns and Matching

Welcome to Part 2! Now that you understand the fundamentals, let's dive into the pattern system - **the most powerful feature of SyftBox permissions**.

**The Problem This Solves**: In real projects, you don't want to set permissions on individual files one by one. You have:
- Hundreds of Python files that should all have the same access rules
- Documentation files scattered across multiple directories  
- Test files in various subdirectories that need special permissions
- Different file types (images, configs, scripts) requiring different access levels

**The Solution**: SyftBox's pattern system lets you write rules like `*.py` (all Python files) or `docs/**/*.md` (all Markdown files anywhere under docs). One pattern can control permissions for hundreds or thousands of files.

**Why This Matters**: Without patterns, managing permissions in large projects would be impossible. With patterns, you can create sophisticated permission schemes with just a few rules that automatically apply to new files as they're created.

### What You'll Learn
- **Glob pattern syntax** (`*`, `**`, `?`) - The building blocks of powerful rules
- **Pattern specificity scoring** - How SyftBox resolves conflicts when multiple patterns match
- **Double-star (`**`) recursive matching** - The most powerful pattern for nested directories
- **Debugging pattern matches** - Essential skills for troubleshooting when patterns don't work as expected

### Prerequisites
- Completed Part 1: Fundamentals
- Understanding of basic file/directory patterns (helpful but not required)

In [ ]:
import syft_perm as sp
from pathlib import Path
import tempfile
import shutil

# Setup workspace
tutorial_dir = Path(tempfile.mkdtemp(prefix="patterns_tutorial_"))
print(f"Tutorial workspace: {tutorial_dir}")

def cleanup_and_reset():
    global tutorial_dir
    if tutorial_dir.exists():
        shutil.rmtree(tutorial_dir)
    tutorial_dir = Path(tempfile.mkdtemp(prefix="patterns_tutorial_"))
    print(f"Fresh workspace: {tutorial_dir}")
    return tutorial_dir

def show_yaml(path):
    yaml_file = path / "syft.pub.yaml"
    if yaml_file.exists():
        print(f"\n=== {yaml_file} ===")
        print(yaml_file.read_text())
        print("=" * 50)

def test_pattern_match(pattern, test_files, expected_matches):
    """Helper to test if a pattern matches expected files"""
    print(f"\n=== Testing pattern: '{pattern}' ===")
    
    for i, (file_path, should_match) in enumerate(zip(test_files, expected_matches)):
        syft_file = sp.open(file_path)
        has_access = syft_file.has_read_access("testuser@example.com")
        
        status = "✓" if has_access == should_match else "✗ WRONG"
        match_text = "MATCH" if should_match else "NO MATCH"
        actual_text = "HAS ACCESS" if has_access else "NO ACCESS"
        
        rel_path = file_path.relative_to(tutorial_dir)
        print(f"{status} {rel_path} - Expected: {match_text}, Got: {actual_text}")

## Chapter 1: Basic Glob Patterns

**The Problem**: You have a project with dozens or hundreds of files of different types - Python scripts, documentation, configuration files, images, etc. You want to give your team different access to different file types:
- Developers should access Python files
- Technical writers should access documentation 
- DevOps engineers should access configuration files
- Everyone should read README files

Setting permissions on each individual file would take hours and be impossible to maintain.

**The Solution**: Glob patterns let you match groups of files by name patterns. Instead of listing 50 Python files individually, you write `*.py` once and it matches all Python files automatically - including ones that don't exist yet!

Let's start with the fundamental pattern types:

In [ ]:
# Create a variety of files to test patterns
files_to_create = [
    "document.txt",
    "image.png", 
    "script.py",
    "README.md",
    "config.json",
    "test_file.py",
    "backup.txt.bak",
    "data.csv"
]

test_files = []
for filename in files_to_create:
    file_path = tutorial_dir / filename
    file_path.write_text(f"Content of {filename}")
    test_files.append(file_path)

print("Created test files:")
for f in test_files:
    print(f"  {f.name}")

### Single Asterisk (*) - Matches any characters except /

**Real-world scenario**: Your development team needs access to all Python files for code review, but you don't want to list every single `.py` file individually.

**The `*` wildcard is your solution** - it matches any sequence of characters except directory separators (`/`). This makes it perfect for matching file extensions or name patterns within a single directory.

In [ ]:
# REAL-WORLD EXAMPLE: Development team needs access to all Python files
cleanup_and_reset()

# Recreate a realistic project file mix
test_files = []
for filename in files_to_create:
    file_path = tutorial_dir / filename
    file_path.write_text(f"Content of {filename}")
    test_files.append(file_path)

print("🏗️ Realistic Project Files:")
for f in test_files:
    print(f"  {f.name}")

print("\n💼 SCENARIO: Development team needs code review access to Python files")
print("❌ Bad approach: Grant access to script.py, test_file.py individually")
print("✅ Good approach: Use pattern '*.py' to match ALL Python files")

# Create permission rule with *.py pattern
yaml_file = tutorial_dir / "syft.pub.yaml"
yaml_content = """rules:
- pattern: "*.py"
  access:
    read:
    - dev_team@company.com
"""
yaml_file.write_text(yaml_content)

print("\n🎯 Pattern: '*.py' means 'any filename ending in .py'")
print("   - script.py ✓ matches")
print("   - test_file.py ✓ matches") 
print("   - document.txt ✗ doesn't match (not .py)")
print("   - image.png ✗ doesn't match (not .py)")

# Expected matches: script.py and test_file.py should match
expected = [
    False,  # document.txt
    False,  # image.png
    True,   # script.py ✓
    False,  # README.md
    False,  # config.json
    True,   # test_file.py ✓
    False,  # backup.txt.bak
    False   # data.csv
]

test_pattern_match("*.py", test_files, expected)

print("\n💡 Key Benefits:")
print("✨ Works for files that don't exist yet - new .py files automatically get permissions")
print("🔧 Maintainable - one rule instead of dozens of individual file permissions")  
print("🎯 Precise - only matches .py files, not other types")

### Question Mark (?) - Matches exactly one character

**Real-world scenario**: You have data files numbered sequentially (data1.txt, data2.txt, etc.) and want to give access to single-digit numbered files but not the double-digit ones (which might be more sensitive).

**The `?` wildcard is perfect for this** - it matches exactly one character, giving you precise control over filename patterns.

In [ ]:
# Create files with single character variations
cleanup_and_reset()

single_char_files = [
    "data1.txt",
    "data2.txt", 
    "data10.txt",  # Two digits - should NOT match
    "dataA.txt",
    "data.txt",    # No character - should NOT match
]

test_files = []
for filename in single_char_files:
    file_path = tutorial_dir / filename
    file_path.write_text(f"Content of {filename}")
    test_files.append(file_path)

# Pattern: data?.txt
yaml_content = """rules:
- pattern: "data?.txt"
  access:
    read:
    - testuser@example.com
"""
(tutorial_dir / "syft.pub.yaml").write_text(yaml_content)

print("Pattern: 'data?.txt' should match files with exactly one character between 'data' and '.txt'")

expected = [
    True,   # data1.txt ✓
    True,   # data2.txt ✓  
    False,  # data10.txt (two characters)
    True,   # dataA.txt ✓
    False,  # data.txt (no character)
]

test_pattern_match("data?.txt", test_files, expected)

## Chapter 2: Double Asterisk (**) - Recursive Matching

**The Problem**: Real projects have deeply nested directory structures. You might have documentation scattered across many subdirectories:
- `docs/user/guide.md`
- `docs/dev/api/endpoints.md`  
- `docs/internal/processes/security.md`

Using single asterisk patterns would require many rules: `docs/*.md`, `docs/user/*.md`, `docs/dev/*.md`, `docs/dev/api/*.md`, etc. This becomes unmanageable quickly.

**The Solution**: The `**` pattern is the most powerful feature in SyftBox patterns. It matches **any number of directories recursively**, allowing you to reach into arbitrarily deep directory structures with a single rule.

**When You Need This**: 
- Any time you have nested directories
- When you want to match file types "anywhere under" a certain directory
- For project-wide rules that should apply regardless of directory structure

Let's see the double asterisk in action:

In [ ]:
# Create nested directory structure
cleanup_and_reset()

# Structure:
# tutorial_dir/
#   ├── docs/
#   │   ├── guide.md
#   │   └── api/
#   │       └── reference.md
#   ├── src/
#   │   ├── main.py
#   │   └── utils/
#   │       └── helper.py
#   └── README.md

nested_files = {
    "README.md": tutorial_dir / "README.md",
    "docs/guide.md": tutorial_dir / "docs" / "guide.md",
    "docs/api/reference.md": tutorial_dir / "docs" / "api" / "reference.md", 
    "src/main.py": tutorial_dir / "src" / "main.py",
    "src/utils/helper.py": tutorial_dir / "src" / "utils" / "helper.py"
}

for rel_path, full_path in nested_files.items():
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(f"Content of {rel_path}")

print("Created nested structure:")
for rel_path in nested_files.keys():
    print(f"  {rel_path}")

In [ ]:
# Test pattern: **/*.md (all Markdown files at any depth)
yaml_content = """rules:
- pattern: "**/*.md"
  access:
    read:
    - testuser@example.com
"""
(tutorial_dir / "syft.pub.yaml").write_text(yaml_content)

print("\nPattern: '**/*.md' should match all .md files at any depth")

test_files = list(nested_files.values())
expected = [
    True,   # README.md ✓
    True,   # docs/guide.md ✓
    True,   # docs/api/reference.md ✓
    False,  # src/main.py
    False,  # src/utils/helper.py
]

test_pattern_match("**/*.md", test_files, expected)

In [ ]:
# Test pattern: docs/** (everything under docs/)
yaml_content = """rules:
- pattern: "docs/**"
  access:
    read:
    - testuser@example.com
"""
(tutorial_dir / "syft.pub.yaml").write_text(yaml_content)

print("\nPattern: 'docs/**' should match everything under docs/ directory")

expected = [
    False,  # README.md (not under docs/)
    True,   # docs/guide.md ✓
    True,   # docs/api/reference.md ✓
    False,  # src/main.py
    False,  # src/utils/helper.py
]

test_pattern_match("docs/**", test_files, expected)

## Chapter 3: Pattern Specificity and Conflicts

**The Problem**: In complex projects, a single file might match multiple patterns. Consider this file: `docs/guide.md`

It could match:
- `**` (matches everything)
- `*.md` (matches all Markdown files) 
- `docs/*` (matches files directly in docs/)
- `docs/guide.md` (exact match)

**Which rule should apply?** Without a clear resolution system, permissions would be unpredictable and debugging would be impossible.

**The Solution**: SyftBox uses a **specificity scoring system** that automatically chooses the most specific rule. Think of it like CSS - more specific selectors override general ones.

**Why This Matters**: 
- Predictable behavior - you can reason about which rule will apply
- Flexible architecture - you can set broad defaults and specific overrides
- Easy debugging - the most specific rule always wins

Let's see specificity in action with a realistic conflict scenario:

In [ ]:
# Create a file that could match multiple patterns
cleanup_and_reset()

docs_file = tutorial_dir / "docs" / "guide.md"
docs_file.parent.mkdir(parents=True)
docs_file.write_text("Documentation guide")

print(f"Created: {docs_file.relative_to(tutorial_dir)}")
print("\nThis file could match multiple patterns:")
print("  - **           (matches everything - least specific)")
print("  - *.md         (matches all .md files)")
print("  - docs/*       (matches files directly in docs/)")
print("  - docs/guide.md (exact match - most specific)")

In [ ]:
# Test pattern conflicts - most specific wins
yaml_content = """rules:
- pattern: "**"
  access:
    read:
    - everyone@example.com
- pattern: "*.md"
  access:
    read:
    - md_readers@example.com
- pattern: "docs/*"
  access:
    read:
    - docs_team@example.com
- pattern: "docs/guide.md"
  access:
    read:
    - guide_editors@example.com
"""
(tutorial_dir / "syft.pub.yaml").write_text(yaml_content)

print("\nTesting pattern specificity (most specific wins):")
syft_docs = sp.open(docs_file)

# Only the most specific pattern should apply
users_to_test = [
    ("everyone@example.com", "**", False),
    ("md_readers@example.com", "*.md", False), 
    ("docs_team@example.com", "docs/*", False),
    ("guide_editors@example.com", "docs/guide.md", True),  # Most specific wins!
]

for user, pattern, should_have_access in users_to_test:
    has_access = syft_docs.has_read_access(user)
    status = "✓" if has_access == should_have_access else "✗ WRONG"
    print(f"{status} {user} (pattern: {pattern}): {has_access}")

print("\nThe most specific pattern 'docs/guide.md' wins!")

### Understanding Specificity Scoring

SyftBox calculates specificity using this formula:
```
score = 2 * literal_chars + 10 * directory_depth - wildcard_penalties
```

In [ ]:
# Let's manually calculate specificity scores for our patterns
patterns = {
    "**": "Universal wildcard - lowest score",
    "*.md": "Extension wildcard", 
    "docs/*": "Directory + wildcard",
    "docs/guide.md": "Exact match - highest score"
}

print("=== Pattern Specificity Analysis ===")
print("Formula: 2 * literal_chars + 10 * directory_depth - wildcard_penalties")
print()

for pattern, description in patterns.items():
    print(f"Pattern: '{pattern}' ({description})")
    
    # Count components
    literal_chars = len([c for c in pattern if c not in ['*', '?']])
    directory_depth = pattern.count('/')
    wildcards = pattern.count('*') + pattern.count('?')
    
    # Simple approximation of the scoring
    approx_score = 2 * literal_chars + 10 * directory_depth - wildcards
    
    print(f"  Literal chars: {literal_chars}, Depth: {directory_depth}, Wildcards: {wildcards}")
    print(f"  Approx score: {approx_score}")
    print()

## Chapter 4: Advanced Double-Star Patterns

Double-star patterns can be combined in powerful ways.

In [ ]:
# Create complex nested structure for advanced testing
cleanup_and_reset()

complex_files = {
    "src/main/java/App.java": "Java app",
    "src/main/python/app.py": "Python app", 
    "src/test/java/AppTest.java": "Java test",
    "src/test/python/test_app.py": "Python test",
    "docs/dev/java/guide.md": "Java dev guide",
    "docs/dev/python/guide.md": "Python dev guide",
    "docs/user/manual.md": "User manual",
    "config/dev/database.json": "Dev DB config",
    "config/prod/database.json": "Prod DB config"
}

test_files = []
for rel_path, content in complex_files.items():
    full_path = tutorial_dir / rel_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(content)
    test_files.append(full_path)

print("Created complex structure:")
for rel_path in complex_files.keys():
    print(f"  {rel_path}")

In [ ]:
# Test pattern: src/**/java/**
# Should match anything under any 'java' directory within src/
yaml_content = """rules:
- pattern: "src/**/java/**"
  access:
    read:
    - java_devs@example.com
"""
(tutorial_dir / "syft.pub.yaml").write_text(yaml_content)

print("\nPattern: 'src/**/java/**' should match Java files in src/")

expected = [
    True,   # src/main/java/App.java ✓
    False,  # src/main/python/app.py
    True,   # src/test/java/AppTest.java ✓
    False,  # src/test/python/test_app.py
    False,  # docs/dev/java/guide.md (not under src/)
    False,  # docs/dev/python/guide.md
    False,  # docs/user/manual.md
    False,  # config/dev/database.json
    False,  # config/prod/database.json
]

test_pattern_match("src/**/java/**", test_files, expected)

In [ ]:
# Test pattern: **/dev/**
# Should match anything under any 'dev' directory anywhere
yaml_content = """rules:
- pattern: "**/dev/**"
  access:
    read:
    - developers@example.com
"""
(tutorial_dir / "syft.pub.yaml").write_text(yaml_content)

print("\nPattern: '**/dev/**' should match anything under any 'dev' directory")

expected = [
    False,  # src/main/java/App.java
    False,  # src/main/python/app.py
    False,  # src/test/java/AppTest.java
    False,  # src/test/python/test_app.py
    True,   # docs/dev/java/guide.md ✓
    True,   # docs/dev/python/guide.md ✓
    False,  # docs/user/manual.md
    True,   # config/dev/database.json ✓
    False,  # config/prod/database.json
]

test_pattern_match("**/dev/**", test_files, expected)

## Chapter 5: Multiple Rules and First-Match Wins

When a file matches multiple patterns **within the same directory**, the **first matching rule** (highest specificity) wins.

In [ ]:
# Test multiple rules in same directory
cleanup_and_reset()

test_file = tutorial_dir / "important.py"
test_file.write_text("Important Python script")

# Create multiple rules - order matters!
yaml_content = """rules:
- pattern: "important.py"
  access:
    admin:
    - admin@example.com
- pattern: "*.py"
  access:
    read:
    - developers@example.com
- pattern: "**"
  access:
    read:
    - everyone@example.com
"""
(tutorial_dir / "syft.pub.yaml").write_text(yaml_content)

print("Testing first-match-wins within same directory:")
print("File 'important.py' matches all three patterns, but first (most specific) wins")

syft_test = sp.open(test_file)

# Only the first (most specific) rule should apply
print(f"\nAdmin has admin access: {syft_test.has_admin_access('admin@example.com')}")
print(f"Developers have read access: {syft_test.has_read_access('developers@example.com')}")
print(f"Everyone has read access: {syft_test.has_read_access('everyone@example.com')}")

# Show reasoning
has_read, reasons = syft_test._check_permission_with_reasons("admin@example.com", "read")
print(f"\nWhy admin has read access: {reasons}")

## Chapter 6: Pattern Debugging Techniques

**The Problem**: Patterns don't always work as expected, especially when you're new to the system. Common frustrations:
- "I wrote `docs/api/*.md` but it doesn't match `docs/api/v1/endpoints.md` - why not?"
- "My pattern should match this file but permissions aren't working"
- "Two patterns seem to conflict - which one is actually being used?"

**The Solution**: SyftBox includes powerful debugging tools that show you exactly WHY a pattern matched or didn't match. Instead of guessing, you can trace the algorithm step-by-step.

**Essential Debugging Skills**: 
- Using `_check_permission_with_reasons()` to see the algorithm's reasoning
- Understanding common pattern mistakes (like `*` vs `**`)
- Testing patterns incrementally from simple to complex

Let's walk through a realistic debugging scenario:

In [ ]:
# Create a scenario where patterns might not work as expected
cleanup_and_reset()

# User expects this pattern to match, but it doesn't
problem_file = tutorial_dir / "docs" / "api" / "v1" / "endpoints.md"
problem_file.parent.mkdir(parents=True)
problem_file.write_text("API documentation")

# Problematic pattern - user thinks this should work
yaml_content = """rules:
- pattern: "docs/api/*.md"
  access:
    read:
    - api_team@example.com
"""
(tutorial_dir / "syft.pub.yaml").write_text(yaml_content)

print("=== Pattern Debugging Example ===")
print(f"File: {problem_file.relative_to(tutorial_dir)}")
print(f"Pattern: 'docs/api/*.md'")
print(f"Expected: Should match")

syft_problem = syft_perm.open(problem_file)
has_access = syft_problem.has_read_access("api_team@example.com")
print(f"Actual: {has_access}")
print(f"\n❌ BUG: Pattern doesn't match! Why?")

In [ ]:
# Debug step 1: Check the reasoning
has_read, reasons = syft_problem._check_permission_with_reasons("api_team@example.com", "read")
print("=== Debugging Step 1: Check Reasons ===")
print(f"Has access: {has_read}")
print("Reasons:")
for reason in reasons:
    print(f"  - {reason}")

print("\n💡 Analysis: 'No permission found' means the pattern didn't match")

In [ ]:
# Debug step 2: Understand why the pattern doesn't match
print("=== Debugging Step 2: Pattern Analysis ===")
print(f"File path: docs/api/v1/endpoints.md")
print(f"Pattern:   docs/api/*.md")
print()
print("Problem: The '*' wildcard doesn't match across directories!")
print("'*' matches 'v1/endpoints' but there's a '/' in between")
print()
print("Solutions:")
print("1. Use 'docs/api/**/*.md' to match any depth under api/")
print("2. Use 'docs/api/*/*.md' to match exactly one level under api/")
print("3. Use 'docs/api/v1/*.md' for exact path matching")

In [ ]:
# Fix the pattern
yaml_content = """rules:
- pattern: "docs/api/**/*.md"
  access:
    read:
    - api_team@example.com
"""
(tutorial_dir / "syft.pub.yaml").write_text(yaml_content)

print("=== Fixed Pattern ===")
print(f"New pattern: 'docs/api/**/*.md'")

# Test the fix
syft_fixed = sp.open(problem_file)
has_access = syft_fixed.has_read_access("api_team@example.com")
print(f"Now works: {has_access} ✓")

# Verify reasoning
has_read, reasons = syft_fixed._check_permission_with_reasons("api_team@example.com", "read")
print(f"Reason: {reasons[0]}")

## Chapter 7: Common Pattern Pitfalls

Learn from common mistakes!

In [ ]:
print("=== Common Pattern Pitfalls ===")
print()

pitfalls = [
    {
        "mistake": "Using * across directories",
        "wrong": "docs/*.md",
        "file": "docs/api/guide.md", 
        "right": "docs/**/*.md",
        "why": "* doesn't match / characters"
    },
    {
        "mistake": "Forgetting file extensions",
        "wrong": "scripts/*",
        "file": "scripts/deploy.sh",
        "right": "scripts/**",
        "why": "* matches everything in directory"
    },
    {
        "mistake": "Case sensitivity issues",
        "wrong": "*.PDF",
        "file": "document.pdf",
        "right": "*.pdf",
        "why": "Patterns are case-sensitive"
    },
    {
        "mistake": "Over-specific patterns",
        "wrong": "src/main/java/com/example/App.java",
        "file": "src/main/java/com/mycompany/App.java",
        "right": "src/**/App.java",
        "why": "Too specific, doesn't handle variations"
    }
]

for i, pitfall in enumerate(pitfalls, 1):
    print(f"{i}. {pitfall['mistake']}")
    print(f"   ❌ Wrong: '{pitfall['wrong']}' won't match '{pitfall['file']}'")
    print(f"   ✅ Right: '{pitfall['right']}'")
    print(f"   💡 Why: {pitfall['why']}")
    print()

## Chapter 8: Pattern Testing Worksheet

Practice with these exercises!

In [ ]:
# Create test files for exercises
cleanup_and_reset()

exercise_files = {
    "README.md": "Root readme",
    "src/main.py": "Main Python file",
    "src/utils/helper.py": "Helper utilities",
    "tests/test_main.py": "Main tests",
    "tests/unit/test_utils.py": "Unit tests",
    "docs/guide.md": "User guide",
    "docs/api/endpoints.md": "API documentation",
    "config/dev.json": "Dev configuration",
    "config/prod.json": "Prod configuration",
    "static/css/style.css": "Stylesheets",
    "static/js/app.js": "JavaScript",
    "data/input.csv": "Input data",
    "data/processed/output.csv": "Processed data"
}

for rel_path, content in exercise_files.items():
    full_path = tutorial_dir / rel_path
    full_path.parent.mkdir(parents=True, exist_ok=True)
    full_path.write_text(content)

print("Exercise files created:")
for rel_path in exercise_files.keys():
    print(f"  {rel_path}")

print("\n=== Pattern Exercises ===")
print("Try to write patterns that match:")
print("1. All Python files anywhere")
print("2. All files in the 'src' directory and subdirectories")
print("3. All test files (files with 'test' in the name)")
print("4. All configuration files (*.json in config/)")
print("5. All documentation files (*.md anywhere)")
print("\nAnswers in the next cell...")

In [ ]:
# Exercise answers
print("=== Exercise Answers ===")
print()

exercises = [
    ("All Python files anywhere", "**/*.py", ["src/main.py", "src/utils/helper.py", "tests/test_main.py", "tests/unit/test_utils.py"]),
    ("All files in 'src' directory", "src/**", ["src/main.py", "src/utils/helper.py"]),
    ("All test files", "**/test*.py", ["tests/test_main.py", "tests/unit/test_utils.py"]),
    ("All config JSON files", "config/*.json", ["config/dev.json", "config/prod.json"]),
    ("All documentation files", "**/*.md", ["README.md", "docs/guide.md", "docs/api/endpoints.md"])
]

for i, (description, pattern, expected_matches) in enumerate(exercises, 1):
    print(f"{i}. {description}")
    print(f"   Pattern: '{pattern}'")
    print(f"   Should match: {', '.join(expected_matches)}")
    print()
    
    # Test the pattern
    yaml_content = f"""rules:
- pattern: "{pattern}"
  access:
    read:
    - testuser@example.com
"""
    (tutorial_dir / "syft.pub.yaml").write_text(yaml_content)
    
    print(f"   Testing pattern '{pattern}':")
    for rel_path in exercise_files.keys():
        full_path = tutorial_dir / rel_path
        syft_file = sp.open(full_path)
        has_access = syft_file.has_read_access("testuser@example.com")
        
        if has_access:
            status = "✓" if rel_path in expected_matches else "✗ UNEXPECTED"
            print(f"     {status} {rel_path}")
    print()

## Summary: Part 2 Key Concepts

Excellent work! You've mastered SyftBox pattern matching:

### ✅ **Pattern Types Mastered**
1. **Single asterisk (*)**: Matches any characters except `/`
2. **Question mark (?)**: Matches exactly one character
3. **Double asterisk (**)**: Recursive matching across directories
4. **Literal text**: Exact string matching

### ✅ **Key Rules Learned**
1. **Specificity matters**: More specific patterns win over general ones
2. **First match wins**: Within same directory, highest specificity rule applies
3. **Case sensitive**: Patterns must match exact case
4. **Path separators**: `*` doesn't cross `/` boundaries, use `**` instead

### ✅ **Debugging Skills**
1. Use `_check_permission_with_reasons()` to see why patterns match/don't match
2. Test patterns incrementally from simple to complex
3. Watch out for common pitfalls (case, directory boundaries)

### 🚀 **Next Up: Part 3 - Inheritance and Hierarchy**
In the next tutorial, you'll learn:
- How permissions flow through directory trees
- The nearest-node algorithm in detail
- Complex multi-level inheritance scenarios
- When inheritance stops and starts

Patterns + Inheritance = The full power of SyftBox permissions!

In [ ]:
# Cleanup
shutil.rmtree(tutorial_dir)
print("Tutorial 2 complete! 🎉")
print("Ready for Part 3: Inheritance and Hierarchy")